# A very simple power system model tutorial

This notebook provides a very simple tutorial into some of the functionality found in the models in this repository. The models are simple: they meet demand with four generation technologies: baseload (cf. nuclear), peaking (cf. gas), wind, and solar.

This notebook is only intended as a very quick introduction. For more extensive documentation, including the optimisation problem that these models solve, see the `documentation` directory.

In [ ]:
# Suppress minor warnings
import warnings
warnings.filterwarnings('ignore')

In [ ]:
import models

# One region model

We start with a model with just one region, viewing the whole system as a "copper plate".

### Import time series data

In [ ]:
ts_data = models.load_time_series_data('1_region')
ts_data = ts_data.loc['2017-06-08':'2017-06-15']
ts_data.head(10)

### `Operate` mode: meet demand with a fixed system

In this example, the generation capacities are user-defined, and the model optimises how demand is met. This is sometimes called the *economic dispatch* or the *unit commitment* problem.

In [ ]:
# Set the generation capacities
generation_capacities = {'cap_baseload_total': 15.0,
                         'cap_peaking_total': 10.0,
                         'cap_wind_total': 20.0,
                         'cap_solar_total': 15.0}  # GW

# Create the model with the fixed capacities
model = models.OneRegionModel(ts_data=ts_data,
                              run_mode='operate',
                              fixed_caps=generation_capacities)

In [ ]:
# Run the model, solve the optimisation problem
model.run()

In [ ]:
# Show a summary of outputs
model.get_summary_outputs()

In [ ]:
# Change colors (for the plot only) 
model._model_data.colors.loc['peaking'] = '#888888'
model._model_data.colors.loc['baseload'] = '#003399'
model._model_data.colors.loc['solar'] = '#e3bb12'
model._model_data.colors.loc['demand_power'] = '#000000'
model._model_data.colors.loc['unmet'] = '#000000'
model._model_data.colors.loc['wind'] = '#4eba44'

# Note: these plots do not display well in Github -- use your own notebook
model.plot.timeseries(array='all')

In [ ]:
# Export all model outputs to CSV (creates directory called 'outputs_operate')
model.to_csv('outputs_operate')

## `Plan` mode: design cost-optimal system

In this example, the model optimises both the design and operation of a system. This is sometimes called the *capacity expansion planning* problem.

In [ ]:
# Create the model without any fixed capacities
model = models.OneRegionModel(ts_data=ts_data,
                              run_mode='plan')

In [ ]:
# Run the model, solve the optimisation problem
model.run()

In [ ]:
# Show a summary of outputs
model.get_summary_outputs()

In [ ]:
# Change colors (for the plot only) 
model._model_data.colors.loc['peaking'] = '#888888'
model._model_data.colors.loc['baseload'] = '#003399'
model._model_data.colors.loc['solar'] = '#e3bb12'
model._model_data.colors.loc['demand_power'] = '#000000'
model._model_data.colors.loc['unmet'] = '#000000'
model._model_data.colors.loc['wind'] = '#4eba44'

# Note: these plots do not display well in Github -- use your own notebook
model.plot.timeseries(array='all')

In [ ]:
# Export all model outputs to CSV (creates directory called 'outputs_plan')
model.to_csv('outputs_plan')

# 6 region model

<img align="right" src="documentation/6_region_diagram.jpg" alt="drawing" width="300" height="250">

We can do the same with a model with 6 regions. In this system, supply and demand must match across the model as a whole but electricity may be transmitted around the grid according to a topology inspired by the *IEEE 6-bus test system* and [Kamalinia & Shahidehpour (2010)](https://doi.org/10.1049/iet-gtd.2009.0695). The regions contain the following demand and generation technologies:
- Region 1: baseload & peaking generation
- Region 2: demand, wind and solar generation, with time series from Germany
- Region 3: baseload & peaking generation
- Region 4: demand, with time series from France
- Region 5: demand, wind and solar generation, with time series from the United Kingdom
- Region 6: baseload, peaking, wind and solar generation, with time series from Spain

Transmission is permitted between regions 1-2, 1-5, 1-6, 2-3, 3-4, 4-5 and 5-6.

### Load the time series data

In [ ]:
ts_data = models.load_time_series_data('6_region')
ts_data = ts_data.loc['2017-06-07':'2017-06-15']
ts_data[ts_data < 0] = 0
ts_data.head(10)

### Run the model in `plan` mode

In [ ]:
model = models.SixRegionModel(ts_data=ts_data,
                              run_mode='plan')
model.run()
model.get_summary_outputs()

In [ ]:
# Note: these plots do not display well in Github -- use your own notebook
model.plot.timeseries(array='all')